In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import datetime

pd.options.mode.chained_assignment = None  # default='warn'


# Load the dataset
file_path = "__sprint_1_alt_no_mans_sky_labelled_steam_review_data.csv"
df = pd.read_csv(file_path, encoding='latin1', low_memory=False)

#heuristic
#informative
#df.head(10)

wip_df = df.drop(columns=[#'recommendationid'
                          #,'clean_sentence'
                          #,'tokenized_sentence'
                          #,'voted_up'
                          'language'
                          ,'timestamp_created'
                          ,'timestamp_updated'
                          ,'votes_up'
                          ,'votes_funny'
                          ,'weighted_vote_score'
                          ,'steam_purchase'
                          ,'author_num_games_owned'
                          ,'author_num_reviews'
                          ,'author_playtime_forever'
                          ,'author_last_played'
                          #,'informative'
                          #,'heuristic'
                 ])
#wip_df.head(10)
informative_df = wip_df.drop(columns=['recommendationid', 'tokenized_sentence', 'voted_up', 'heuristic'])
heuristic_df = wip_df.drop(columns=['recommendationid', 'tokenized_sentence', 'voted_up','informative'])
#informative_df.head(10)
heuristic_df.head(10)

,clean_sentence,heuristic
0,every planet moon remarkably unique,G
1,simple exploration grants things encounter sce...,G
2,well worth buy like explore advance,G
3,note graphics game runs like crap 980ti even f...,U
4,welp foundations update seems fixed lot stuff ...,F
5,avatar walks snails pace sprint ten seconds re...,G
6,huge improvements great game,NaN
7,game sort goal maybe monsters bosses fight,G
8,next question might could game made better sure,NaN
9,base building lots vehicles land air sea along...,G


In [3]:
wip_df.columns

Index(['recommendationid', 'clean_sentence', 'tokenized_sentence', 'voted_up',
       'informative', 'heuristic'],
      dtype='object')

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

#Playability informative Naives Bayes Classification

X = informative_df[(informative_df['informative'].notnull()) & (informative_df['informative'] != '')]
X = X.dropna()
y = X['informative']
X = X.drop(columns=['informative'])
#print(X.count())
#print(y.count())

# Convert the text data to a bag-of-words representation
vectorizer = CountVectorizer()
texts = X['clean_sentence']
X = vectorizer.fit_transform(texts)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

# Initialize the Naive Bayes classifier (MultinomialNB is well-suited for count data)
classifier = MultinomialNB()

# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.8
Classification Report:
               precision    recall  f1-score   support

         0.0       0.82      0.80      0.81       298
         1.0       0.78      0.80      0.79       267

    accuracy                           0.80       565
   macro avg       0.80      0.80      0.80       565
weighted avg       0.80      0.80      0.80       565



In [5]:
print(informative_df['informative'].value_counts())
print(informative_df['clean_sentence'].count())

informative
0.0    1527
1.0    1332
Name: count, dtype: int64
515471


In [6]:
#Execution started at 16:37

In [8]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

27.Feb 2025 15:51:07


In [9]:
print(informative_df['informative'].value_counts())

for row in informative_df[(informative_df['informative'].isnull()) | (informative_df['informative'] == '') ].itertuples(index=True, name='Row'):
    #print(row)
    #if len(row.clean_sentence):
    vect = vectorizer.transform([str(row.clean_sentence)])
    informative_df.loc[row.Index, 'informative'] = classifier.predict(vect)
    #else:
    #    informative_df.loc[row.Index, 'informative'] = 0

for row in wip_df[(wip_df['informative'].isnull()) | (wip_df['informative'] == '') ].itertuples(index=True, name='Row'):
    vect = vectorizer.transform([str(row.clean_sentence)])
    wip_df.loc[row.Index, 'informative'] = classifier.predict(vect)

print(informative_df['informative'].value_counts())

informative
0.0    1527
1.0    1332
Name: count, dtype: int64
informative
0.0    295928
1.0    227720
Name: count, dtype: int64


In [11]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

27.Feb 2025 15:57:44


In [12]:
#informative_df[(informative_df['informative'] == 0)]['clean_sentence'].tail(50)

In [13]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

27.Feb 2025 15:57:44


In [14]:
heuristic_df.head(20)
# 'heuristic'
heuristic_training = heuristic_df.loc[(heuristic_df['heuristic'].notnull()) & (heuristic_df['heuristic'] != '')]

#Tidy up the label values
for row in heuristic_training.itertuples(index=True, name='Row'):
    heuristic_training.loc[row.Index, 'heuristic'] = str(heuristic_training.loc[row.Index, 'heuristic']).replace('"','').replace("'",'').replace(' ','').replace('[','').replace(']','').split(',')



heuristic_training.dropna(how='any')
heuristic_training.describe()

#heuristic_training.head(20)
heuristic_training['heuristic'].value_counts()

heuristic
[U]          320
[G]          278
[F]          272
[G, U]        62
[F, U]        28
[G, F]        22
[U, F]        10
[F, G, U]      6
[G, F, U]      5
[F, G]         4
[U, G]         4
[G, U, F]      1
[F, U, G]      1
[U, G, F]      1
Name: count, dtype: int64

In [15]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

27.Feb 2025 15:57:45


In [16]:
print(len(heuristic_training))

1014


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier

# Encode multi-labels
mlb = MultiLabelBinarizer()
label_matrix = mlb.fit_transform(heuristic_training['heuristic'])

# Convert text data into numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(heuristic_training['clean_sentence'])



In [18]:
heuristic_training['heuristic'].value_counts()

heuristic
[U]          320
[G]          278
[F]          272
[G, U]        62
[F, U]        28
[G, F]        22
[U, F]        10
[F, G, U]      6
[G, F, U]      5
[F, G]         4
[U, G]         4
[G, U, F]      1
[F, U, G]      1
[U, G, F]      1
Name: count, dtype: int64

In [19]:
len(heuristic_training)

1014

In [20]:
label_matrix

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [21]:
mlb.inverse_transform(np.array([[1, 1, 1]]))

[('F', 'G', 'U')]

In [23]:
mlb.classes_

array(['F', 'G', 'U'], dtype=object)

In [24]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
# Define cross-validation strategy (5-fold)
kf = KFold(n_splits=5, shuffle=True, random_state=1)

# Store F1 scores for each fold
f1_scores = []

for k in range(3, 20):
    # Perform cross-validation
    for fold, (train_index, test_index) in enumerate(kf.split(X_tfidf)):
        # Split data into training and testing sets
        X_train, X_test = X_tfidf[train_index], X_tfidf[test_index]
        y_train, y_test = label_matrix[train_index], label_matrix[test_index]
    
        # Train Binary Relevance model with KNN
        br_knn = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=k))
        br_knn.fit(X_train, y_train)
    
        # Predict on test data
        y_pred = br_knn.predict(X_test)
    
        # Compute micro-average F1-score for multi-label classification
        f1 = f1_score(y_test, y_pred, average="micro", zero_division=0)
        f1_scores.append(f1)
    
        #print(f"k={k}: Fold {fold+1} - Micro F1-score: {f1:.4f}")

    # Compute average F1-score across all folds
    avg_f1_score = np.mean(f1_scores)
    print(f"k={k}: Average Micro F1-score across all folds: {avg_f1_score:.4f}")

k=3: Average Micro F1-score across all folds: 0.6859
k=4: Average Micro F1-score across all folds: 0.6558
k=5: Average Micro F1-score across all folds: 0.6696
k=6: Average Micro F1-score across all folds: 0.6687
k=7: Average Micro F1-score across all folds: 0.6783
k=8: Average Micro F1-score across all folds: 0.6759
k=9: Average Micro F1-score across all folds: 0.6815
k=10: Average Micro F1-score across all folds: 0.6809
k=11: Average Micro F1-score across all folds: 0.6852
k=12: Average Micro F1-score across all folds: 0.6852
k=13: Average Micro F1-score across all folds: 0.6881
k=14: Average Micro F1-score across all folds: 0.6884
k=15: Average Micro F1-score across all folds: 0.6906
k=16: Average Micro F1-score across all folds: 0.6907
k=17: Average Micro F1-score across all folds: 0.6923
k=18: Average Micro F1-score across all folds: 0.6926
k=19: Average Micro F1-score across all folds: 0.6944


In [25]:
#Train the model after identifying the best k=3
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, label_matrix, test_size=0.2, random_state=1
)

# Train Binary Relevance model with KNN
br_knn = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=3))
br_knn.fit(X_train, y_train)

# Predict on test data
y_pred = br_knn.predict(X_test)

# Evaluate model performance
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0))

# Convert predictions back to label format
y_pred_labels = mlb.inverse_transform(y_pred)
print("\nPredicted Labels:\n", y_pred_labels)

Classification Report:
               precision    recall  f1-score   support

           F       0.80      0.72      0.76        72
           G       0.77      0.58      0.66        79
           U       0.72      0.70      0.71        82

   micro avg       0.76      0.67      0.71       233
   macro avg       0.76      0.67      0.71       233
weighted avg       0.76      0.67      0.71       233
 samples avg       0.72      0.69      0.69       233


Predicted Labels:
 [('F',), ('F',), ('U',), ('U',), ('G',), ('G', 'U'), ('F',), ('G',), ('G',), (), ('F',), ('F',), ('G',), ('F',), ('F',), ('G',), ('U',), ('U',), ('F',), ('U',), ('U',), ('G',), ('F',), ('G',), ('U',), ('G',), ('U',), ('F',), ('F',), ('F',), ('F',), ('F',), ('F',), ('G',), ('G',), (), ('U',), ('G',), ('G', 'U'), ('F',), ('U',), ('G',), ('G',), (), ('F', 'G'), ('U',), ('U',), ('G',), ('F',), ('U',), ('F',), ('U',), ('U',), ('U',), ('F',), ('F',), ('U',), ('F',), ('F', 'U'), ('G',), ('G',), (), ('U',), ('G',), ('U',), 

In [26]:
informative_df.tail(10)

,clean_sentence,informative
523638,sometimes trade sell things offer buy visiting...,1.0
523639,trust know software engineer,0.0
523640,recharging hyperdrive probably biggest challen...,1.0
523641,good game,0.0
523642,would love play game unless enjoy staring load...,1.0
523643,homogenous resource availability,1.0
523644,yep got screwed,0.0
523645,please fix happy work done hello games also ch...,0.0
523646,NaN,0.0
523647,list features leave original release mere shad...,0.0



#Tidy up the label values
for row in wip_df.itertuples(index=True, name='Row'):
    
    tempVal = wip_df.loc[row.Index, 'heuristic']
    print(tempVal)
    if type(tempVal) is list:
        asd=0 #Do nothing
    elif type(tempVal) is float:
        wip_df.loc[row.Index, 'heuristic'] = []
    elif type(tempVal) is str:
        wip_df.loc[row.Index, 'heuristic'] = []
    elif len(tempVal)==0:
        wip_df.loc[row.Index, 'heuristic'] = []
    elif tempVal.isnull():
        wip_df.loc[row.Index, 'heuristic'] = []
    else:
        wip_df.loc[row.Index, 'heuristic'] = list(str(tempVal).replace(',','').replace("'",'').replace(' ',''))
    
    

In [29]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

27.Feb 2025 15:57:46


In [30]:
pd.options.mode.chained_assignment = None  # default='warn'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print(len(wip_df['heuristic']))
for row in wip_df.itertuples(index=True, name='Row'):
    #print(row.Index)
    #print(wip_df['heuristic'][row.Index] )
    if type(wip_df['heuristic'][row.Index]) is str:
        tempVal = list(str(wip_df['heuristic'][row.Index]).replace(',','').replace("'",'').replace(' ',''))
        if len(tempVal)==0:
            asd = 0
            #wip_df['heuristic'][row.Index] = []
        else:
            #wip_df.loc[row.Index, 'heuristic'] = tempVal
            wip_df['heuristic'][row.Index] = tempVal
    #elif type(wip_df['heuristic'][row.Index]) is list:
        #Do nothing
        #asd=0
    #elif type(wip_df['heuristic'][row.Index]) is float:
        #Do nothing
        #asd=0
        #wip_df['heuristic'][row.Index] = []
    #elif wip_df['heuristic'][row.Index].isnull() or wip_df['heuristic'][row.Index] == '':
        #wip_df['heuristic'][row.Index] = []
    #else:
        #wip_df['heuristic'][row.Index] = []


wip_df['heuristic'].value_counts()

523648


heuristic
[U]          320
[G]          278
[F]          272
[G, U]        62
[F, U]        28
[G, F]        22
[U, F]        10
[F, G, U]      6
[G, F, U]      5
[F, G]         4
[U, G]         4
[G, U, F]      1
[F, U, G]      1
[U, G, F]      1
Name: count, dtype: int64

In [33]:
wip_df['informative'].value_counts()

informative
0.0    295928
1.0    227720
Name: count, dtype: int64

In [32]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

27.Feb 2025 15:57:54


In [34]:
backup = wip_df

In [35]:
print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

27.Feb 2025 15:57:54


In [36]:


#for row in wip_df[(wip_df['informative'] == 1) & (wip_df['heuristic'].isnull()) ].itertuples(index=True, name='Row'):
for row in wip_df[(wip_df['informative'] == 1) ].itertuples(index=True, name='Row'):
    vect1 = vectorizer.transform([str(row.clean_sentence)])
    #print(wip_df[(wip_df['informative'] == 1) & (wip_df['heuristic'].isnull()) ].loc[row.Index, 'heuristic'])
    #print(br_knn.predict(vect1))
    #print(mlb.inverse_transform(br_knn.predict(vect1)))
    #print(row.Index)
    wip_df['heuristic'][row.Index] = mlb.inverse_transform(br_knn.predict(vect1))
    




In [37]:

print(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S"))

27.Feb 2025 16:23:58


In [38]:
wip_df['heuristic'].value_counts()

heuristic
[(G,)]         102342
[(U,)]          49969
[(F,)]          36936
[()]            19088
[(G, U)]        11484
[(F, G)]         4436
[(F, U)]         3005
[(F, G, U)]       460
[F]                21
[U]                19
[G, U]              6
[F, U]              2
[G]                 2
[F, G, U]           1
[U, F]              1
[G, F]              1
Name: count, dtype: int64

In [39]:
wip_df['informative'].value_counts()

informative
0.0    295928
1.0    227720
Name: count, dtype: int64

In [40]:
mlb.classes_

array(['F', 'G', 'U'], dtype=object)

In [45]:
wip_df['heuristic'].value_counts()

heuristic
[(G,)]         102342
[(U,)]          49969
[(F,)]          36936
[()]            19088
[(G, U)]        11484
[(F, G)]         4436
[(F, U)]         3005
[(F, G, U)]       460
[F]                21
[U]                19
[G, U]              6
[F, U]              2
[G]                 2
[F, G, U]           1
[U, F]              1
[G, F]              1
Name: count, dtype: int64

In [113]:
# making boolean series for a team name
#g_filter = 'G' in  wip_df['heuristic']

tuple_df = wip_df[wip_df['heuristic'].apply(lambda x: isinstance(x, list))]

g_set = tuple_df[tuple_df['heuristic'].apply(lambda x: 'G' in x[0])]
u_set = tuple_df[tuple_df['heuristic'].apply(lambda x: 'U' in x[0])]
f_set = tuple_df[tuple_df['heuristic'].apply(lambda x: 'F' in x[0])]

#g_set['heuristic'].value_counts()

# filtering data
#g_set = wip_df.where(g_filter)



In [117]:
g_set['heuristic'].value_counts()

heuristic
[(G,)]         102342
[(G, U)]        11484
[(F, G)]         4436
[(F, G, U)]       460
[G, U]              6
[G]                 2
[G, F]              1
Name: count, dtype: int64

In [119]:
u_set['heuristic'].value_counts()

heuristic
[(U,)]         49969
[(G, U)]       11484
[(F, U)]        3005
[(F, G, U)]      460
[U]               19
[U, F]             1
Name: count, dtype: int64

In [121]:
f_set['heuristic'].value_counts()

heuristic
[(F,)]         36936
[(F, G)]        4436
[(F, U)]        3005
[(F, G, U)]      460
[F]               21
[F, U]             2
[F, G, U]          1
Name: count, dtype: int64

In [135]:
g_set.to_csv('__sprint1_filtered_gameplay_review_set.csv')
f_set.to_csv('__sprint1_filtered_functionality_review_set.csv')
u_set.to_csv('__sprint1_filtered_usability_review_set.csv')